#### prevendo vendas de videogames

tratando os dados

In [163]:
from tensorflow.keras.layers import Dense, Dropout, Activation, Input
from tensorflow.keras.models import Model
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
import pandas as pd

In [164]:
base = pd.read_csv('games.csv')

In [165]:
base

,Name,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Critic_Score,Critic_Count,User_Score,User_Count,Developer,Rating
0,Wii Sports,Wii,2006.0,Sports,Nintendo,41.36,28.96,3.77,8.45,82.53,76.0,51.0,8,322.0,Nintendo,E
1,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24,NaN,NaN,NaN,NaN,NaN,NaN
2,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.68,12.76,3.79,3.29,35.52,82.0,73.0,8.3,709.0,Nintendo,E
3,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.61,10.93,3.28,2.95,32.77,80.0,73.0,8,192.0,Nintendo,E
4,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16714,Samurai Warriors: Sanada Maru,PS3,2016.0,Action,Tecmo Koei,0.00,0.00,0.01,0.00,0.01,NaN,NaN,NaN,NaN,NaN,NaN
16715,LMA Manager 2007,X360,2006.0,Sports,Codemasters,0.00,0.01,0.00,0.00,0.01,NaN,NaN,NaN,NaN,NaN,NaN
16716,Haitaka no Psychedelica,PSV,2016.0,Adventure,Idea Factory,0.00,0.00,0.01,0.00,0.01,NaN,NaN,NaN,NaN,NaN,NaN
16717,Spirits & Spells,GBA,2003.0,Platform,Wanadoo,0.01,0.00,0.00,0.00,0.01,NaN,NaN,NaN,NaN,NaN,NaN


In [166]:
base = base.drop(['Other_Sales', 'Global_Sales', 'Developer',], axis=1)

In [167]:
base.shape

(16719, 13)

In [168]:
base.isnull().sum()

Name                  2
Platform              0
Year_of_Release     269
Genre                 2
Publisher            54
NA_Sales              0
EU_Sales              0
JP_Sales              0
Critic_Score       8582
Critic_Count       8582
User_Score         6704
User_Count         9129
Rating             6769
dtype: int64

In [169]:
# no video o instrutor apenas exlui os dados nulos
# como são muitos nulos, eu não vou só apagar as linhas dos valores numerico
# ao inves, irei pegar a media

# preenchendo valores nulos nas colunas numéricas com a média
numeric_columns = base.select_dtypes(include=['float64', 'int64']).columns
base[numeric_columns] = base[numeric_columns].apply(lambda x: x.fillna(x.mean()))


In [170]:
base.isnull().sum()

Name                  2
Platform              0
Year_of_Release       0
Genre                 2
Publisher            54
NA_Sales              0
EU_Sales              0
JP_Sales              0
Critic_Score          0
Critic_Count          0
User_Score         6704
User_Count            0
Rating             6769
dtype: int64

In [171]:
base = base.dropna(axis=0)

In [172]:
base.shape

(9873, 13)

In [173]:
base['Name'].value_counts()
# o nome não ajudará a rede a generalizar valores
base = base.drop('Name', axis=1)

In [174]:
base.shape

(9873, 12)

In [175]:
base.columns

Index(['Platform', 'Year_of_Release', 'Genre', 'Publisher', 'NA_Sales',
       'EU_Sales', 'JP_Sales', 'Critic_Score', 'Critic_Count', 'User_Score',
       'User_Count', 'Rating'],
      dtype='object')

In [178]:
# na coluna user_score há dados não numericos vou exlui-los 
import numpy as np
print(np.sum(base['User_Score'] == 'tbd'))
base = base[base['User_Score'] != 'tbd']

2374


In [180]:
X = base.iloc[:, [0,1,2,3,7,8,9,10,11]].values
y_na = base.iloc[:, 4].values
y_eu = base.iloc[:, 5].values
y_jp = base.iloc[:, 6].values

In [181]:
print(X)

[['Wii' 2006.0 'Sports' ... '8' 322.0 'E']
 ['Wii' 2008.0 'Racing' ... '8.3' 709.0 'E']
 ['Wii' 2009.0 'Sports' ... '8' 192.0 'E']
 ...
 ['PC' 2014.0 'Action' ... '7.6' 412.0 'M']
 ['PC' 2011.0 'Shooter' ... '5.8' 43.0 'T']
 ['PC' 2011.0 'Strategy' ... '7.2' 13.0 'E10+']]


In [182]:
base['Platform'].value_counts()

# como não há hierarquia entre as categorias
# serão criadas 17 novas colunas
# 0000000000000001
# 0000000000000010
# 0000000000000100
# 0000000000001000
# ...



Platform
PS2     1240
X360     958
PS3      865
PC       725
XB       585
Wii      559
DS       515
PSP      421
GC       366
GBA      255
PS4      246
XOne     176
3DS      173
PS       158
PSV      143
WiiU     100
DC        14
Name: count, dtype: int64

In [183]:


ohe = ColumnTransformer(
    transformers=[("OneHot", OneHotEncoder(), [0, 2, 3, 8])], 
    remainder='passthrough'
)



In [184]:
X = ohe.fit_transform(X)

In [185]:
X.shape

(7499, 313)

criando a rede neural

In [189]:
camada_entrada = Input(shape=(313,))
camada_oculta1 = Dense(units=int((313+3)/2), activation='relu')(camada_entrada)
camada_oculta2 = Dense(units=int((313+3)/2), activation='relu')(camada_oculta1)
camada_saida1 = Dense(units=1, activation='linear')(camada_oculta2)
camada_saida2 = Dense(units=1, activation='linear')(camada_oculta2)
camada_saida3 = Dense(units=1, activation='linear')(camada_oculta2)

regressor = Model(inputs=camada_entrada, outputs=[camada_saida1, camada_saida2, camada_saida3])


In [190]:
regressor.compile(optimizer='adam', loss='mse') #mean_squared_error da mais peso a erros grandes

In [191]:
regressor.fit(X, [y_na, y_eu, y_jp], 
              epochs=500, 
              batch_size=100)

Epoch 1/500
75/75 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - dense_29_loss: 450.8465 - dense_30_loss: 138.8792 - dense_31_loss: 3274.0071 - loss: 3863.7371  
Epoch 2/500
75/75 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - dense_29_loss: 7.3728 - dense_30_loss: 4.7930 - dense_31_loss: 4.9715 - loss: 17.1373
Epoch 3/500
75/75 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - dense_29_loss: 2.3622 - dense_30_loss: 3.5157 - dense_31_loss: 1.5438 - loss: 7.4217
Epoch 4/500
75/75 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - dense_29_loss: 1.3162 - dense_30_loss: 1.1993 - dense_31_loss: 0.3678 - loss: 2.8833
Epoch 5/500
75/75 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - dense_29_loss: 1.0993 - dense_30_loss: 1.3732 - dense_31_loss: 0.4410 - loss: 2.9135
Epoch 6/500
75/75 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - dense_29_loss: 1.6011 - dense_30_loss: 1.4346 - dense_31_loss: 0.3093 - loss: 3.3451
Epoch 7/500
75/75 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - dense_29_loss: 1.8993 - dense_30_loss: 8.9500 - dense_31_loss: 2.1994 - loss: 13.0483
Epoch 8/500
75/75 ━━

In [193]:
previsao_na, previsao_eu, previsao_jp= regressor.predict(X)

235/235 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step


In [198]:
previsao_na.mean()

0.41433623

In [200]:
y_na.mean()

0.37977463661821576

In [202]:
from sklearn.metrics import mean_absolute_error

In [203]:
mean_absolute_error(y_na, previsao_na)

0.30429159963348734

In [206]:
mean_absolute_error(y_eu, previsao_eu)

0.21096344389268787

In [207]:
mean_absolute_error(y_jp, previsao_jp)

0.09928818126474481